In [3]:
from speech_to_text import SpeechToText
stt = SpeechToText()


/home/duy1332002/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
stt.print_result("/home/duy1332002/Desktop/Lip_Reading_THDH/data/speech_to_text/test7.wav")

 cho đến một ngày khi ngồi từ tốn viết xuống một thứ gì đó gọi là


In [4]:
stt.save_result_to_file('/home/duy1332002/Desktop/Lip_Reading_THDH/data/speechtotext/t5.wav', '/home/duy1332002/Desktop/Lip_Reading_THDH/data/speechtotext/test1.txt')

Save result to file:  /home/duy1332002/Desktop/Lip_Reading_THDH/data/speechtotext/test1.txt


# Correction

In [4]:
import soundfile as sf
import pandas as pd
import torch
import numpy as np
import IPython
import pvleopard
from underthesea import word_tokenize

In [22]:
def check_alignment(audio_path, alignment_path, save_audio=False, path_save_audio=None):
    '''
    check the result of alignment can be predict by speech to text or not
    parameter:
        audio_path: str - path audio
        alignment_path: str - path csv
        save_audio: bool - save audio or not
        path_save_audio: str - path save audio, if save_audio is True, you should give the path
    return:
        pandas table - the alignment is correct or not
    '''
    # check the audio file is exist or not
    if not os.path.exists(audio_path):
        print('File not found')
        return None
    # load dummy dataset and read soundfiles
    ds = stt.map_to_array({"file": audio_path})
    ds['speech'] = np.mean(ds['speech'], axis = 1)

    # read alignment csv
    df_alignment = pd.read_csv(alignment_path, names=['start', 'end', 'word'])
    df_result = pd.DataFrame(columns=['start', 'end', 'word', 'predict'])

    t = 1 / ds['sampling_rate']

    audio_list =  []
    for i in range(len(df_alignment)):
        start = int((df_alignment['start'].values[i] / t))
        end = int((df_alignment['end'].values[i] / t))
        word_sound = ds['speech'][start:end]
        word_predict = stt._transcribe_with_vector(word_sound, ds)
        if save_audio:
            if not os.path.exists(path_save_audio):
                os.mkdir(path_save_audio)
            sf.write(path_save_audio + '/' + df_alignment['word'].values[i] + '.wav', word_sound, ds['sampling_rate'])
        df_result = pd.concat([df_result, pd.DataFrame([[df_alignment['start'].values[i], 
        df_alignment['end'].values[i], df_alignment['word'].values[i], word_predict]], 
        columns=['start', 'end', 'word', 'predict'])], ignore_index=True)
    return df_result

In [23]:
k = check_alignment('test7.wav', 'test7.csv', save_audio=True, path_save_audio='test7')

In [24]:
k

,start,end,word,predict
0,0.336,0.528,cho,
1,0.624,0.720,đến,
2,0.768,1.008,một,hai
3,1.056,1.248,ngày,
4,1.296,1.440,khi,
5,1.488,1.680,ngồi,hồi
6,1.728,1.920,từ,
7,1.968,2.208,tốn,
8,2.256,2.496,viết,
9,2.544,2.784,xuống,


In [27]:
def second_to_timecode(x: float) -> str:
    hour, x = divmod(x, 3600)
    minute, x = divmod(x, 60)
    second, x = divmod(x, 1)
    millisecond = int(x * 1000.)
    return '%.2d:%.2d:%.2d,%.3d' % (hour, minute, second, millisecond)

In [29]:
def convert_to_srt(alignement_path):
    df = pd.read_csv(alignement_path, names=['start', 'end', 'word'])
    with open('test.srt', 'w') as f:
        for i in range(len(df)):
            f.write(f'{i+1}\n')
            f.write(f'{second_to_timecode(df["start"].values[i])} --> {second_to_timecode(df["end"].values[i])}\n')
            f.write(f'{df["word"].values[i]}\n\n')
convert_to_srt('test7.csv')


# Count words

In [2]:
import os
import pandas as pd
transcripts = r'..\..\data\transcripts'
word_dict = dict()
trans_lst = os.listdir(transcripts)
for i in range(28):
    transcript_path = os.path.join(transcripts, trans_lst[i])
    with open(transcript_path, 'r', encoding='utf-8') as f:
        words = f.read().split()
        for word in words:
            word_dict[word] = word_dict.get(word, 0) + 1
df = pd.DataFrame(columns=['word', 'count'])
for word, count in word_dict.items():
    new_word = pd.DataFrame({'word': [word],
                             'count': [count]})
    df = pd.concat([df, new_word], ignore_index=True)
sorted_df = df.sort_values(by=['count'], ascending=False)
sorted_df.to_csv('stat.csv', index=False)
print(len(sorted_df))

1683


In [26]:
from speech_to_text import SpeechToText
import os


def transcribe_and_align(speech2text,
                         video_name,
                         audios_path,
                         transcripts_path,
                         csv_transcripts_path):
    '''
        Transcribe the audio and align each word to the speech
    '''
    audio_name = video_name.replace('mp4', 'wav')
    audio_path = os.path.join(audios_path, audio_name)
    transcript_path = os.path.join(transcripts_path,
                                   video_name.replace('mp4', 'txt'))
    csv_transcript_path = os.path.join(
        csv_transcripts_path,
        video_name.replace('mp4', 'csv')
    )

    if not os.path.isfile(csv_transcript_path):
        print(f'Transcribing and aligning {audio_name}')
        speech2text.save_result_to_file(audio_path,
                                        transcript_path,
                                        csv_transcript_path)


def convert_to_srt(video_name,
                   transcripts_path,
                   csv_transcripts_path,
                   srt_transcripts_path):
    '''
        Convert csv file to srt file
    '''

    def sec_to_timecode(x: float) -> str:
        '''
            Calculate timecode from second

            Parameters:
                x: float
                    Second
        '''
        hour, x = divmod(x, 3600)
        minute, x = divmod(x, 60)
        second, x = divmod(x, 1)
        millisecond = int(x * 1000.)
        return '%.2d:%.2d:%.2d,%.3d' % (hour, minute, second, millisecond)

    csv_name = video_name.replace('mp4', 'csv')
    csv_path = os.path.join(
        csv_transcripts_path,
        csv_name
    )
    srt_name = video_name.replace('mp4', 'srt')
    srt_path = os.path.join(
        srt_transcripts_path,
        srt_name
    )

    if not os.path.isfile(srt_path):
        df = pd.read_csv(csv_path)

        print(f'Converting {csv_name} to {srt_name}')
        with open(srt_path, 'w', encoding='utf-8') as f:
            for i in range(len(df)):
                start = df["start"].values[i]
                end = df["end"].values[i]
                word = df["word"].values[i]
                f.write(f'{i+1}\n')
                f.write(
                    f'{sec_to_timecode(start)} --> {sec_to_timecode(end)}\n')
                f.write(f'{word}\n\n')


video_name = r'20230114.mp4'
audios_path = r'..\..\test_data\audios'
transcripts_path = r'..\..\test_data\transcripts'
csv_transcripts_path = r'..\..\test_data\csv_transcripts'
srt_transcripts_path = r'..\..\test_data\srt_transcripts'
speech2text = SpeechToText()

transcribe_and_align(speech2text, video_name, audios_path,
                     transcripts_path, csv_transcripts_path)
convert_to_srt(video_name, transcripts_path,
               csv_transcripts_path, srt_transcripts_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing and aligning 20230114.wav
Save transcript to file:  ..\..\test_data\transcripts\20230114.txt
Save transcript to file:  ..\..\test_data\csv_transcripts\20230114.csv
Converting 20230114.csv to 20230114.srt


# Trim silience

In [4]:
import ffmpeg
import pandas as pd

def trim_and_extract_audio(raw_videos_path,
                           video_name,
                           videos_path,
                           audios_path,
                           start=60, end=None) -> None:
    '''
        Trim a part of the video and then extract audio

        Parameters:
            start: float
                where to begin trimming
            end: float
                where to end trimming
    '''
    raw_video_path = os.path.join(raw_videos_path, video_name)
    video_path = os.path.join(videos_path, video_name)
    audio_path = os.path.join(audios_path,
                                video_name.replace('mp4', 'wav'))
    video_stream, audio_stream = __get_video_stream(
        raw_video_path, start, end)
    # Trim first minute of the video
    if not os.path.isfile(video_path):
        print(f'Trimming {video_name}')
        video_output = ffmpeg.output(ffmpeg.concat(video_stream,
                                                    audio_stream,
                                                    v=1, a=1),
                                        video_path,
                                        format='mp4')
        video_output.run()
    # Extract audio
    if not os.path.isfile(audio_path):
        print(f'Extracting audio from {video_name}')
        audio_output = ffmpeg.output(audio_stream,
                                        audio_path,
                                        format='wav')
        audio_output.run()

def __get_video_stream(raw_video_path: str, start, end) -> tuple:
    '''
        Get video stream and audio stream

        Parameters:
            raw_video_path: str
                Path to video
            start: float
                Second in video to start streaming
            end: float
                Second in video to end streaming
    '''
    input_stream = ffmpeg.input(raw_video_path)
    pts = 'PTS-STARTPTS'
    if end is None:
        end = (ffmpeg
                .probe(raw_video_path)
                .get('format', {})
                .get('duration'))    # Get video's length
    video = (input_stream
                .trim(start=start, end=end)
                .filter('setpts', pts))
    audio = (input_stream
                .filter('atrim', start=start, end=end)
                .filter('asetpts', pts))
    return video, audio

def transcribe_and_align(speech2text, video_name, audios_path, transcripts_path, csv_transcripts_path):
    '''
        Transcribe the audio and align each word to the speech
    '''
    audio_name = video_name.replace('mp4', 'wav')
    audio_path = os.path.join(audios_path, audio_name)
    transcript_path = os.path.join(transcripts_path,
                                    video_name.replace('mp4', 'txt'))
    csv_transcript_path = os.path.join(
        csv_transcripts_path,
        video_name.replace('mp4', 'csv')
    )

    if not os.path.isfile(csv_transcript_path):
        print(f'Transcribing and aligning {audio_name}')
        speech2text.save_result_to_file(audio_path,
                                                transcript_path,
                                                csv_transcript_path)

def convert_to_srt(video_name, csv_transcripts_path, srt_transcripts_path):
    '''
        Convert csv file to srt file
    '''

    def sec_to_timecode(x: float) -> str:
        '''
            Calculate timecode from second

            Parameters:
                x: float
                    Second
        '''
        hour, x = divmod(x, 3600)
        minute, x = divmod(x, 60)
        second, x = divmod(x, 1)
        millisecond = int(x * 1000.)
        return '%.2d:%.2d:%.2d,%.3d' % (hour, minute, second, millisecond)

    csv_name = video_name.replace('mp4', 'csv')
    csv_path = os.path.join(
        csv_transcripts_path,
        csv_name
    )
    srt_name = video_name.replace('mp4', 'srt')
    srt_path = os.path.join(
        srt_transcripts_path,
        srt_name
    )

    if not os.path.isfile(srt_path):
        df = pd.read_csv(csv_path)

        print(f'Converting {csv_name} to {srt_name}')
        with open(srt_path, 'w', encoding='utf-8') as f:
            for i in range(len(df)):
                start = df["start"].values[i]
                end = df["end"].values[i]
                word = df["word"].values[i]
                f.write(f'{i+1}\n')
                f.write(
                    f'{sec_to_timecode(start)} --> {sec_to_timecode(end)}\n')
                f.write(f'{word}\n\n')

In [7]:
from speech_to_text import SpeechToText

video_name = r'no_silent_moment_nottrim.mp4'
raw_videos_path = r'..\..\test_data\raw_videos'
videos_path = r'..\..\test_data\videos'
audios_path = r'..\..\test_data\audios'
transcripts_path = r'..\..\test_data\transcripts'
csv_transcripts_path = r'..\..\test_data\csv_transcripts'
srt_transcripts_path = r'..\..\test_data\srt_transcripts'
speech2text = SpeechToText()

trim_and_extract_audio(raw_videos_path, video_name, videos_path, audios_path)
transcribe_and_align(speech2text, video_name, audios_path, transcripts_path, csv_transcripts_path)
convert_to_srt(video_name, csv_transcripts_path, srt_transcripts_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing and aligning no_silent_moment_nottrim.wav
Save transcript to file:  ..\..\test_data\transcripts\no_silent_moment_nottrim.txt
Save transcript to file:  ..\..\test_data\csv_transcripts\no_silent_moment_nottrim.csv
Converting no_silent_moment_nottrim.csv to no_silent_moment_nottrim.srt


In [ ]:
srt_name = '20230113.srt'
with open(srt_name, 'r', encoding='utf-8') as f:
    print(f.read())